In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [143]:
train = pd.read_csv('./data/Train_Fyxd0t8.csv', index_col='ID')
test = pd.read_csv('./data/Test_C1XBIYq.csv', index_col='ID')

In [163]:
%run scripts/features.py
%run scripts/eval.py

In [145]:
# impute missing values
train = train.fillna(train.Number_Weeks_Used.mean())
test = test.fillna(test.Number_Weeks_Used.mean())

In [146]:
y = train.Crop_Damage

In [147]:
from sklearn.cross_validation import train_test_split

In [148]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=44)

In [149]:
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import xgboost as xgb

In [170]:
ft = FeatureTransformer()
# scaler = MinMaxScaler()
# clf = LogisticRegression(C=0.001)
# clf = KNeighborsClassifier()
# clf = RandomForestClassifier()
clf = xgb.XGBClassifier(n_estimators=300, learning_rate=0.08, max_depth=6, min_child_weight=3, subsample=0.8, colsample_bytree=0.8)

pipeline = Pipeline([('ft', ft), ('clf', clf)])

In [171]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('ft', FeatureTransformer()), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.08, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=300, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])

In [172]:
# cross validation
scores = eval_models([pipeline], X_train, y_train)

accuracy score: 0.845196
combined score: 0.845196
accuracy score: 0.847025
combined score: 0.847025
accuracy score: 0.845337
combined score: 0.845337
accuracy score: 0.840132
combined score: 0.840132
accuracy score: 0.844141
combined score: 0.844141


In [173]:
print 'Mean Accuracy score %f ' %(np.mean(scores))

Mean Accuracy score 0.844366 


In [174]:
predsTest = pipeline.predict(X_test)
print 'Accuracy score on the test examples ', classification_report(y_test, predsTest)

Accuracy score on the test examples               precision    recall  f1-score   support

          0       0.86      0.98      0.92     14837
          1       0.52      0.17      0.26      2457
          2       0.20      0.00      0.01       478

avg / total       0.80      0.84      0.80     17772



In [175]:
# confusion matrix
confusion_matrix(y_test, predsTest)

array([[14585,   250,     2],
       [ 2030,   421,     6],
       [  332,   144,     2]])

In [176]:
# fit on the whole dataset
pipeline.fit(train, y)

Pipeline(steps=[('ft', FeatureTransformer()), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.08, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=300, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])

In [177]:
predictions = pipeline.predict(test)

In [178]:
submissions = pd.read_csv('./data/Sample_Submission_Psj3sjG.csv')

In [179]:
submissions['ID'] = test.index.values
submissions['Crop_Damage'] = predictions

In [180]:
submissions.to_csv('./submissions/seventh.csv', index=False)